# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# fetch timeseries, select
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# add calendar columns
ts = pf.calendar(ts)

# finalize timeseries
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,high,low,open,close,volume,adj_close,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.88,204.18,206.38,205.43,121465900.00,182.75,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.37,201.35,204.17,201.72,169632600.00,179.45,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.72,198.86,202.09,199.82,209151400.00,177.76,1,6,6,1,False,False,False,False,False,False
2015-01-07,202.72,200.88,201.42,202.31,125346700.00,179.97,2,7,7,1,False,False,False,False,False,False
2015-01-08,206.16,203.99,204.01,205.90,147217800.00,183.16,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-03,383.70,380.48,382.44,381.85,52427100.00,381.85,2,3,34,2,False,False,False,False,False,False
2021-02-04,386.24,381.97,382.96,386.19,47142600.00,386.19,3,4,35,2,False,False,False,False,False,False
2021-02-05,388.47,386.14,388.20,387.71,48620300.00,387.71,4,5,36,2,False,False,False,True,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy open, sell close
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
314,2021-01-11,377.85,2021-01-11,378.69,0.84,22.68,27,393.51,LONG,SPY
315,2021-01-19,378.34,2021-01-19,378.65,0.31,8.37,27,401.88,LONG,SPY
316,2021-01-25,383.67,2021-01-25,384.39,0.72,19.44,27,421.32,LONG,SPY
317,2021-02-01,373.72,2021-02-01,376.23,2.51,67.77,27,489.09,LONG,SPY
318,2021-02-08,389.27,2021-02-08,390.51,1.24,32.24,26,521.33,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,0.84
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.18
sortino_ratio,0.11
monthly_std,1.46
annual_std,6.93
